# An ensemble method for top-N recommendations from the SVD

## SVD

In [ ]:
%load_ext autotime

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
import pickle
import copy 
import random
import time

In [ ]:
# read training dataset and available item data
data_file = pd.read_table('data/training.csv', sep = ',', header=None, engine='python')
print("Training data shape :", data_file.shape)
movie_file = pd.read_table('ml-1m/movies.dat', sep = '::', header=None, engine='python')
print("Available movie data shape :", movie_file.shape)

In [ ]:
# get user and item list from imported datasets
# maps index to real IDs
users = np.unique(data_file[0])
movies = np.unique(movie_file[0])

#store number of users and items
number_of_rows = len(users)
number_of_columns = len(movies)
print("Number of users :", number_of_rows)
print("Number of items :", number_of_columns)

# map real IDs to indices using dictionary
movie_indices, user_indices = {}, {}
for i in range(len(movies)):
    movie_indices[movies[i]] = i
for i in range(len(users)):
    user_indices[users[i]] = i

In [ ]:
# create sparse matrix
# user on row axis and items on column axis 
# matching rating values on the matrix cells
V = sp.lil_matrix((number_of_rows, number_of_columns))
for line in data_file.values:
    u, i , r , t = map(int,line)
    V[user_indices[u], movie_indices[i]] = r
print("Ratings matrix shape :", V.shape)

In [ ]:
# calculate user and item factors from Singular Value Decomposition
# this might take some time
u,s, vt = svds(V, k = 32)
print("User factors shape :", u.shape)
print("Item factors shape :", vt.shape)

In [ ]:
# map factors into diagonal matrix
s_diag_matrix = np.zeros((s.shape[0], s.shape[0]))
for i in range(s.shape[0]):
    s_diag_matrix[i,i] = s[i]

In [ ]:
# count negativity of user factors
negcounter = 0
poscounter = 0
for i in range(0,s.size):
    for factor in u[:,i]:
        if factor > 0:
            poscounter = poscounter + 1
        else:
            negcounter = negcounter + 1
print("Negative user factors :", negcounter)
print("Positive user factors :" ,poscounter)

In [ ]:
# count negativity of item factors
negcounter = 0
poscounter = 0
for i in range(0,s.size):
    for factor in vt[i,:]:
        if factor > 0:
            poscounter = poscounter + 1
        else:
            negcounter = negcounter + 1
print("Negative item factors :", negcounter)
print("Positive item factors :" ,poscounter)

In [ ]:
# calculate SVD recommendation results for computePrecision
X_lr = np.dot(np.dot(u, s_diag_matrix), vt)

##  Model for Top-N Recommendations

In [ ]:
# Node class declaration
class Node:
    def __init__(self,fsize):
        self.itemFactors = np.empty(shape = (fsize,0))
        self.userFactors = np.empty(shape = (fsize,0))
        self.itemList = []
        self.userList = []
        self.factor = None
        self.factors = []
        self.score = 0
        self.left = None
        self.right = None

In [ ]:
# fill user and item list on Node object with all available users and items
def fillLists(V):
    for user in user_indices:
        V.userList.append(user)
    for item in movie_indices:
        V.itemList.append(item)

In [ ]:
# initiate a full Node object with all available initial data
def restartV():
    V = Node(s.size)
    V.itemFactors = vt
    V.userFactors = u
    fillLists(V)
    factors = []
    factors.extend(range(0,s.size))
    V.factors = factors
    return V

In [ ]:
# check if Node object is a leaf
def isLeaf(node):
    if len(node.itemFactors.shape) < 2:
        return True
    elif node.itemFactors.shape[1] < 300:
        return True
    elif node.factor == None:
        return True
    return False

In [ ]:
# check if the Node object is empty on factors
def isEmpty(node):
    empt = Node(len(node.factors))
    if np.array_equal(node.itemFactors, empt.itemFactors):
        return True
    if np.array_equal(node.userFactors, empt.userFactors):
        return True
    return False

In [ ]:
# prints a Node object with its variables
def printNode(node):
    print("itemFactors shape:") 
    print(node.itemFactors.shape)
    print("userFactors shape:")
    print(node.userFactors.shape)
    print("factor :")
    print(node.factor)
    print("score :")
    print(node.score)
    print("left :")
    print(node.left)
    print("right :")
    print(node.right)

In [ ]:
# print tree structure using printNode function on left and right nodes
def printTree(node):
    printNode(node)
    if (node.left != None):
        print("LEFT :")
        printTree(node.left)
    if (node.right != None):
        print("RIGHT :")
        printTree(node.right)

In [ ]:
# counts leaves on a tree
def countLeaves(node):
    count = 0
    if (node.left != None):
        if isLeaf(node.left):
            count += 1
        else:
            count += countLeaves(node.left)
    else:
        print("ERROR: countLeaves left", node.itemFactors.shape[1])
    if (node.right != None):
        if isLeaf(node.right):
            count += 1
        else:
            count += countLeaves(node.right)
    else:
        print("ERROR countLeaves right", node.itemFactors.shape[1])
    return count

In [ ]:
V = restartV()

In [ ]:
V.itemFactors.shape

In [ ]:
V.userFactors.shape

In [ ]:
# returns top N possible recommendation values
def findTopN(matrix, N):
    newMatrix = matrix.argsort(axis = 1)
    newMatrix = np.fliplr(newMatrix)
    return newMatrix[:,:N] 

In [ ]:
# number of recommendations to provide
precisionAt = 5

# top N items from SVD recommender system
mx = findTopN(X_lr, precisionAt)

# calculates score of Node object from its children split
def computePrecision(node):
    # diagonal node factors matrix
    diag_matrix = np.zeros((len(node.factors), len(node.factors)))
    for i in range(len(node.factors)):
        diag_matrix[i,i] = s_diag_matrix[node.factors,node.factors][i]
    
    # dot product of user factors, diagonal factors matrix and item factors on shild nodes
    relevantRight = np.dot(np.dot(node.right.userFactors,diag_matrix), node.right.itemFactors)
    relevantLeft = np.dot(np.dot(node.left.userFactors,diag_matrix), node.left.itemFactors)
    
    # top N results from dot product calculation
    topRight = findTopN(relevantRight, precisionAt)
    topLeft = findTopN(relevantLeft, precisionAt)
    
    # counters for precision calculation types
    samePlace = 0
    inTopN = 0
    for u in range(len(node.left.userList)):
        for i in range(precisionAt):
            ares = movie_indices[node.left.itemList[topLeft[u,:][i]]]
            amx = mx[user_indices[node.left.userList[u]],:]
            if (ares == amx[i]):
                samePlace += 1
            if ares in amx:
                inTopN += 1
    for u in range(len(node.right.userList)):
        for i in range(precisionAt):
            ares = movie_indices[node.right.itemList[topRight[u,:][i]]]
            amx = mx[user_indices[node.right.userList[u]],:]
            if (ares == amx[i]):
                samePlace += 1
            if ares in amx:
                inTopN += 1
    #print("Recommendations found on exact order :", samePlace)
    #print("Recommendations found present in top N list :", inTopN)
    
    # calculate score by recommendations present in top N
    score = float(inTopN)/(len(node.userList)*precisionAt)
    
    #calculate score by recommendation on same order
    #score = float(samePlace)/(len(node.userList)*precisionAt)
    
    print("Score :",score)
    return score
    

In [ ]:
# splits a Node into right and left Node objects
def splitNode(node, factor):
    factorIndex = node.factors.index(factor)
    
    # initiate left and right Node objects
    node.left = Node(len(node.factors))
    node.right = Node(len(node.factors))
    
    # assigns partial factors into children
    node.left.factors = node.factors
    node.right.factors = node.factors
        
    node.right.userFactors = np.transpose(node.right.userFactors)
    node.left.userFactors = np.transpose(node.left.userFactors)
    node.right.itemFactors = np.transpose(node.right.itemFactors)
    node.left.itemFactors = np.transpose(node.left.itemFactors)
    
    # flag and counter for left and right assigned factors
    left = 0
    right = 0
    
    if not isEmpty(node):
        for i in range(node.itemFactors.shape[1]):
            # factor is positive
            if (node.itemFactors[:, i][factorIndex] >= 0 ):
                if (left == 0):
                    # initiate item factors of left child
                    node.left.itemFactors = node.itemFactors[:, i]
                else:
                    # append item factors of left child
                    node.left.itemFactors = np.vstack((node.left.itemFactors, node.itemFactors[:, i]))
                # append item list of left child
                node.left.itemList.append(node.itemList[i])
                left += 1
            # factor is negative
            else:
                if (right == 0):
                    # initiate item factors of right child
                    node.right.itemFactors = node.itemFactors[:, i]
                else:
                    # append item factors of right child
                    node.right.itemFactors =np.vstack((node.right.itemFactors, node.itemFactors[:, i]))
                # append item list of right child
                node.right.itemList.append(node.itemList[i])
                right += 1
        node.left.itemFactors = np.transpose(node.left.itemFactors)
        node.right.itemFactors = np.transpose(node.right.itemFactors)
    else:
        print("ERROR: itemFactors not available")
        
    # flag and counter for left and right assigned factors
    left = 0
    right = 0
    if not isEmpty(node):
        for i in range(node.userFactors.shape[0]):
            # factor is positive
            if (node.userFactors[i, :][factorIndex] >= 0 ):
                if (left == 0):
                    # initiate user factors of left child
                    node.left.userFactors = node.userFactors[i, :]
                else:
                    # append user factors of left child
                    node.left.userFactors = np.vstack((node.left.userFactors, node.userFactors[i, :]))
                # append user list of left child
                node.left.userList.append(node.userList[i])
                left += 1
            # factor is negative
            else:
                if (right == 0):
                    # initiate user factors of right child
                    node.right.userFactors = node.userFactors[i, :]
                else:
                    # append user factors of right child
                    node.right.userFactors = np.vstack((node.right.userFactors, node.userFactors[i, :]))
                # append user list of right child
                node.right.userList.append(node.userList[i])
                right += 1
    else:
        print("ERROR: userFactors not available")
        
    # calls computePrecision to calculate score for selected factor split
    node.score =  computePrecision(node)

In [ ]:
# builds tree with given partial factors and the root
def buildTree(node, factors):
    print("Partial factor list :", factors)
    # TODO : user yoksa bolunmeyi durdur
    if (len(node.itemFactors.shape) > 1 and node.itemFactors.shape[1] <= 300):
        print("Threshold value is reached")
        return node
    elif (len(node.itemFactors.shape) <= 1):
        print('one item factor')
        return node
    elif (node.itemFactors.shape[1] == 0):
        print('no item factor')
        return node
    winner = Node(len(node.factors))
    if not factors:
        print('factors empty')
        return node
    for factor in factors:
        node.factor = factor
        splitNode(node,factor)
        if (node.score >= winner.score):
            winner = copy.deepcopy(node) 
    #print("Winner split score :", winner.score)
    #print("Winner split factor :", winner.factor)
    
    # remove winner factor from partial factor list
    factors.remove(winner.factor)
    
    # assign winner factor to the root
    node.factor = winner.factor
    
    # partial factor lists for subtrees
    lfactors = list(factors)
    rfactors = list(factors)
    
    # build subtrees with roots of left and right children nodes
    node.left = buildTree(winner.left, lfactors)
    node.right = buildTree(winner.right, rfactors)
    return node
    

In [ ]:
# returns list of partial factor groups
def getFactorGroups(size, p, a):
    groups = []
    for i in range(0,a):
        counterList = np.zeros(len(factors))
        for c in range(0,int(1/p)):
            group = []
            while (len(group) < size * p):
                # list of available factors for the group
                available = []
                for k in range(len(counterList)):
                    if (counterList[k] < 1 and k not in group):
                        available.append(k)
                # get a random factor from available factors
                factor = available[random.randint(0, len(available)-1)]
                counterList[factor] = counterList[factor] + 1 
                group.append(factor)
                
            # sort factor group ascending
            group.sort()
            
            # append group to factor groups list
            groups.append(group)
    return groups
            

In [ ]:
# build forest with given parameters
def buildForest(factors, p, a, sl):
    # initiate empty list for trees in the forest
    forest = []
    
    # get randomized factor groups for each tree
    groups = getFactorGroups(len(factors), p, a)
    
    # generate a tree for each factor group
    for group in groups:
        print(group)
        V = Node(len(group))
        V.itemFactors = vt[group,:]
        V.userFactors = u[:,group]
        fillLists(V)
        V.factors = group
        buildTree(V,group)
        forest.append(V)
    return forest

In [ ]:
# build a single tree or a forest
singleTree = True

# initiate list of factor indices
factors = []
if singleTree:
    V = restartV()
    factors.extend(range(0,s.size))
    #forest.append(buildTree(V,factors))
    forest = [buildTree(V,factors)]
else:
    factors.extend(range(0,s.size))
    forest = buildForest(factors, 0.5, 3, 300)

In [ ]:
print("Number of trees in the forest :", len(forest))
for i in range(len(forest)):
    node = forest[i]
    print("Number of leaves on Tree " + str(i + 1) + " :" , countLeaves(node))

In [ ]:
# print first tree in the forest
printTree(forest[0])

# # Testing Model

In [ ]:
# read test data file
test_file = pd.read_table('data/test.csv', sep = ',', header=None, engine='python')
print("Test dataset shape :", test_file.shape)

In [ ]:
# user list from imported test data
test_users = np.unique(test_file[0])

# number of users on the test dataset
test_number_of_rows = len(test_users)
print("Number of test data users :", test_number_of_rows)

# map real user IDs to indices using dictionary
test_user_indices = {}
for i in range(len(test_users)):
    test_user_indices[test_users[i]] = i

In [ ]:
# create sparse matrix
# user on row axis and items on column axis 
# matching rating values on the matrix cells
test_V = sp.lil_matrix((test_number_of_rows, number_of_columns))
for line in test_file.values:
    test_u, test_i , test_r , test_time = map(int,line)
    if test_i in movie_indices:
        test_V[test_user_indices[test_u], movie_indices[test_i]] = test_r
print("Test ratings matrix shape :", test_V.shape)

In [ ]:
# returns list of recommendation for a given user
def recommend(index):
    # read user factors from offline data
    userFactors = u[index]
    
    # initiate empty top recommendations list
    topList = []
    
    # iterate through each tree in the forest
    for tree in forest:
        node = tree
        # find the corresponding leaf for the user
        while not isLeaf(node):
            if userFactors[node.factor] >= 0:
                if not isEmpty(node.left):
                    node = node.left
                else:
                    break
            else:
                if not isEmpty(node.right):
                    node = node.right
                else:
                    break
        # diagonal factor matrix for the node
        test_diag_matrix = np.zeros((len(node.factors), len(node.factors)))
        for i in range(len(node.factors)):
            test_diag_matrix[i,i] = s_diag_matrix[node.factors,node.factors][i]
            
        # dot product to find relevance values
        relevant = np.dot(np.dot(userFactors[node.factors],test_diag_matrix), node.itemFactors)
        
        # sorted item indices for recommendations
        indexMatrix = relevant.argsort()[::-1]
        
        # watched movie ratings for the user to calculate testing accuracy
        watched = movies[np.nonzero(test_V[index,:])[1]]
        for i in indexMatrix:
            if node.itemList[i] in watched:
                # append item ID and relevance value into top list
                topList.append((node.itemList[i],relevant[i]))
    # sort toplist by relevance values
    topList.sort(key=lambda x: x[1], reverse=True)
    # sorted recommendation list with unique items
    result = []
    for item in topList:
        if not item[0] in result:
            result.append(item[0])
    return result

In [ ]:
# computes user recommendation accuraty by top N recommendations
def computeUserAccuracy(index):
    # get recommendations
    computedMovies = recommend(index)
    
    # return 0 accuracy if recommmendations is empty
    if not computedMovies:
        return 0
    
    # sum of weighted average
    weightedSum = 0
    
    # available recommendations or N
    counter = 0
    if precisionAt > len(computedMovies):
        counter = len(computedMovies) 
    else:
        counter = precisionAt 

    # sum of weights
    sumWeight = (counter * (counter +1)) /2
    
    # iterate through recommendations
    for recommendation in computedMovies:
        if (counter != 0):
            # calculate weighted value
            weightedSum = weightedSum + test_V[index, movie_indices[recommendation]] * counter
            counter = counter - 1 
            
    #return weighted average
    return float(weightedSum / (sumWeight*5))

In [ ]:
# calculates average user accuracy of system
def computeAccuracy():
    # counter for empty recommendations
    empty = 0
    # sum of user accuracies
    sumUserAccuracy = 0.0
    # iterate through every user
    for user in range(0,test_V.shape[0]):
        # get user accuracy
        userAccuracy = computeUserAccuracy(user)
        # empty recommendations
        if (userAccuracy == 0):
            empty = empty + 1
        # add user accuracy to sum of user accuracies
        sumUserAccuracy = sumUserAccuracy + userAccuracy
        # print user accuracy for investigation
        print(userAccuracy)
    # print number of empty recommendations
    print("Number of users with empty recommendations ,", empty)
    
    # average user accuracy
    accuracy = float(sumUserAccuracy / (test_V.shape[0] - empty))
    
    # print average user accuracy of system
    print("Average user accuracy ,", accuracy)
    
    return accuracy

In [ ]:
acc = computeAccuracy()

In [ ]:
computeUserAccuracy(0)

In [ ]:
recommend(0)